In [1]:
import pandas
import numpy
import gcld3
from pycountry import languages
import transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import translators as ts

In [37]:
# returns text's language
# speed or FastText would be better

def detect_language(text):
    detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0, 
                                        max_num_bytes=100000)
    result = detector.FindLanguage(text=text)
    if languages.get(alpha_2=result.language) == None:
        return None
    return languages.get(alpha_2=result.language).name

In [38]:
## translates to English

def translate(text):
    return ts.google('ტექსტი, რომელიც უნდა ითარგმნოს', if_use_cn_host=True)

In [ ]:
TRAIN_PATH = '/Users/test/Downloads/archive/train.ft.txt'
TEST_PATH = '/Users/test/Downloads/archive/test.ft.txt'
xa = 0
cnt = 0
f = open(TRAIN_PATH, "r")
_all = 0
nones = 0
for x in f:
    if x == None:
        nones+=1
        continue
    elif detect_language(x[10:]) != 'English':
        cnt+=1
    _all += 1
        
print('Non English:',cnt)
print('Reviews Number:',_all)
print('Nones:',nones)